___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Bruno Kaczelnik

#### Guilherme Lotaif

####  Renato Tajima

#### Thiago Verardo

___

# Análise dos Airbnb nas maiores cidades americanas

___

## A. INTRODUÇÃO

[Detalhe aqui o objetivo deste projeto!!]

[Aqui mais espaço!]

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [10]:
#Importando bibliotecas:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas_profiling
import seaborn as sns
from scipy.stats import norm,probplot
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas_profiling'

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\PCDELL\Desktop\insper\CDD\projeto 3\Projeto-3-CDD---DPA-master\Projeto-3-CDD---DPA-master


### BASE DE DADOS

**Fonte do dataset:** www.kaggle.com/rudymizrahi/airbnb-listings-in-major-us-cities-deloitte-ml


In [4]:
#Importado o arquivo:
df = pd.read_csv('train.csv')

In [5]:
#Análisando o tamanho do dataframe:
linhas, colunas = df.shape
print("O Dataframe possui {0} linhas por {1} colunas.".format(linhas, colunas))

O Dataframe possui 74111 linhas por 29 colunas.


#### Limpando o dataframe para ser análisado:
Esta etapa consiste em uma preparação do dataframe para facilitar a análise no mesmo, e evitar a ocorrência de complicações ou erros que atrapalhem nossso classificador. Portanto iremos limpar os titulos de cada coluna e remover os valores nulos de cada categoria

In [6]:
#Removendo os espaços em branco dos nomes das colunas:
df.columns = [espaços.strip() for espaços in df.columns.tolist()]

In [7]:
#Apagando valores nulos que podem causar problemas posteriores:
df = df.dropna(axis=0, subset=['bathrooms','first_review','host_has_profile_pic','host_identity_verified',
                               'host_response_rate','host_since','last_review','neighbourhood','review_scores_rating',
                               'thumbnail_url','thumbnail_url','zipcode','bedrooms','beds'])

In [8]:
df.sample(2)

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
36669,20942129,4.430817,Apartment,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",1,1.0,Real Bed,strict,False,...,42.339020,-71.106641,"High-rise room, convenient environs",Fenway/Kenmore,284,97.0,https://a0.muscache.com/im/pictures/90862603/c...,02215,1.0,1.0
47262,11675677,4.248495,Loft,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,strict,True,...,34.120027,-118.190375,GreenSputnik in Highland Park,Highland Park,60,98.0,https://a0.muscache.com/im/pictures/92313f9d-e...,90042,1.0,1.0


#### Transformando categorias qualiativas em quantiativas:

O método usado para efetuar essa conversão é chamado de **One Hot Encoding**, que transforma variaveis categóricas em vetores binarios. Tal método transforma todas as variaveis em 0 menos a do item analisado, desse modo permitindo que sejam feitam análises em cima desse dados.

In [9]:
#Função para chamar o get_dummies e remover a coluna base:
def dummify(data, column_name):
    df1 = data.copy()
    df2 = pd.concat([df1.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

### ANÁLISE DESCRITIVA

Após realizarmos o filtro, deve-se realizar uma análise exploratória dos dados, com o  objetivo de achar as váriaveis que mais influenciam no nosso objetivo e que assim possam nos ajudar a prever qual será a avaliação de um hotel aleatório. Ela será feita com o auxílio do pandas_profiling e seaborn.


In [ ]:
#utilizandoo o pandas_profiling
#df é o dataframe após o filtro
pandas_profiling.ProfileReport(df)

In [ ]:
#Utilizando o seaborn
#df é o dataframe após o filtro

corre = df.corr(method = 'spearman')
sns.heatmap(corre, xticklabels = corre.columns.values, yticklabels = corre.columns.values)

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS

**Fonte do dataset:** www.kaggle.com/rudymizrahi/airbnb-listings-in-major-us-cities-deloitte-ml